In [22]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, render_template, redirect, url_for
from flask_pymongo import PyMongo
import pymongo
import datetime
from webdriver_manager.chrome import ChromeDriverManager

In [23]:
# to setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\lcswi\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


NASA Mars News - Scrape the Mars News Site and collect the latest News Title and Paragraph Text

In [24]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)

In [25]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [26]:
news_title = soup.find_all('div', class_='content_title')[0].text
news_title

"NASA's MAVEN Explores Mars to Understand Radio Interference at Earth"

In [32]:
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
news_p

'NASA’s MAVEN spacecraft has discovered “layers” and “rifts” in the electrically charged part of the upper atmosphere of Mars.'